# セリフを単語ごとに分解し時系列データとみなして分類する
まずはすべてのデータをまとめて単語の出現頻度を確認する、出現頻度の降順に単語を並べた辞書を作成する

In [1]:
import codecs
import numpy as np
import pandas
import random
import sys
from janome.tokenizer import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

#セリフのデータを読みこむ
types = ["cute","cool","passion"]
length = []#何番目までcuteかなどを記録する、[3,5,7]なら0,1,2はcute,3,4はcool,5,6はpassion
serif_all = []
name_all = []
for type_temp in types:#タイプごとにデータを読み込む
    filename = "./data/comment_"+type_temp+".txt"
    file = codecs.open(filename, 'r')
    serif_all.extend(file.readlines())
    length.append(len(serif_all))
    file.close()
    filename = "./data/name_"+type_temp+".txt"
    file = codecs.open(filename,"r")
    name_all.extend(file.readlines())
    file.close()

#品詞ごとに分解する
words = [] # 単語文字列、品詞、登場回数のリスト
t = Tokenizer()
counter = 0
for i in range(len(name_all)):
    name_all[i] = name_all[i].replace("\n","")
    
for serif in serif_all:
    counter += 1
    print(counter,end = "\r")
    serif = serif.replace('\n', '')
    tokens = t.tokenize(serif)#品詞ごとに分解される
    for j in range(0,len(tokens)): #すべての単語についてみていく
        token = tokens[j]#ある単語に注目
        is_new_word = True
        for i in range(len(words)):#既出単語と比較し、tokenが既出か確認
            if words[i][0] == token.surface and words[i][1] == token.part_of_speech[:2]:#tokenの単語そのものと品詞が一致していたら既出とみなす
                words[i][2] += 1#出現回数を増やす
                is_new_word = False
                break
        if is_new_word:#新出単語であれば
            words.append([token.surface, token.part_of_speech[:2], 1])#wordsに記録、part_of_speech[:2]になっているのはpart_of_speechが"名詞,一般,*,*"のような文字列になっているから"
# 単語情報をデータフレームに変換
dic = pandas.DataFrame(words)
dic.columns = ['words', 'parts', 'freq']
dic = dic.sort_values(by=['freq'], ascending=False)
dic = dic.reset_index(drop=True)#出現回数の降順にインデックスを付け直す
num_words = len(dic)
print(dic)

Using TensorFlow backend.


          words parts  freq
0             、    記号  8202
1             …    記号  5063
2             ！    記号  4801
3             の    助詞  4087
4             て    助詞  3968
5             に    助詞  3534
6             。    記号  3463
7             は    助詞  2621
8             ○    記号  2480
9                  記号  2335
10            た    助動  1984
11            が    助詞  1975
12            を    助詞  1949
13            も    助詞  1833
14            ？    記号  1816
15           です    助動  1739
16            で    助詞  1663
17            ね    助詞  1388
18            し    動詞  1372
19            だ    助動  1351
20            よ    助詞  1327
21            と    助詞  1296
22            ー    名詞  1186
23            私    名詞  1168
24            な    助動  1117
25           から    助詞  1115
26            ん    名詞  1048
27            か    助詞  1046
28           ます    助動  1019
29           ない    助動   931
...         ...   ...   ...
10127      思い込ん    動詞     1
10128        区別    名詞     1
10129         熱    形容     1
10130      引っ張り    動

辞書をもとにデータを数字のベクトルに変換する。具体的には、あるセリフについて単語ごとに出現頻度の順位に置き換えていく。  
例：はじめまして、○○プロデューサーさん！
→883 1 9 9 33 31 3  
dataには置き換えられたあとのデータおよびそのタイプやもとのセリフ、キャラクターの情報なども含まれている。

In [2]:
def index_to_type(i,length,types):#インデックスを"cute"などのタイプの文字列に変換する関数
    for j in range(len(length)):
        if i < length[j]:
            return types[j]
            break
# 辞書ができたので全セリフデータを固定長の数字列に変換します
max_speech_length = 100
data_list = []
for i in range(len(serif_all)):
    print(i,end = "\r")
    serif = serif_all[i]
    serif = serif.replace("\n","")
    tokens = t.tokenize(serif)
    record = []#あるセリフの単語がそれぞれ出現回数について何位かを記録する、ついてにタイプも記録
    record.append(index_to_type(i,length,types))#ここのためにiでラベルしている
    n = min(len(tokens),max_speech_length+1)
    for j in range(n):
        dic_temp = dic[(dic['words'] == tokens[j].surface) & (dic['parts'] == tokens[j].part_of_speech[:2])]
        record.append(dic_temp.index[0]+1)#+1しているのは0埋めと区別するため
    if (len(record) < max_speech_length + 1):#max_speech_lengthより品詞数が少ないときには0でうめる
        for j in range(max_speech_length + 1 - len(record)):
            record.append(0)
    record.append(serif)#もとのセリフも
    record.append(name_all[i])
    data_list.append(record)

colnames = ['type']
colnames.extend(range(0, max_speech_length))
colnames.append('original_text')
colnames.append("name") 
data = pandas.DataFrame(data_list, columns=colnames)
print(data)
data.to_csv('./data/data.txt', sep='\t')

         type     0     1     2     3     4     5     6      7     8  \
0        cute   883     1     9     9    33    31     3     10  2137   
1        cute   259   259    33    31     1  2137  1325     16     3   
2        cute  5326     3     2     2   368   556    11      3   999   
3        cute    69     1   122    33     1    81     1     82   378   
4        cute     2  4820  4790    22   945    29     7  10138     4   
5        cute  4820  4790    16     7    24     1    82    254   266   
6        cute   909   271    29     1    33    32  4145   1895     1   
7        cute  1183   296   378    67    16   110     3     10   318   
8        cute   735   770    36    40    13   158     6     60   333   
9        cute  1475    33    31     1    81   804     5    735   770   
10       cute   913   251     1    33    31     1   250     35     4   
11       cute    35     4    34    84     6     8  1080     11    44   
12       cute   740     1    33    31    14   189   114    889  

LSTMについての設定。今回は全く工夫しない

In [27]:
embedding_vecor_length = 64
model = Sequential()
model.add(Embedding(num_words+1, embedding_vecor_length, input_length=max_speech_length))
model.add(Conv1D(64, 3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(Dropout(0.1))
model.add(LSTM(30, dropout_W=0.1, dropout_U=0.1))
model.add(Dropout(0.1))
model.add(Dense(len(types), activation='softmax'))#クラスの数に分類:cute,cool,passion
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  """
C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(30, dropout=0.1, recurrent_dropout=0.1)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 64)           650112    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 50, 64)            0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 50, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 30)                11400     
_________________________________________________________________
dropout_14 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 93        
Total para

学習の前にdataを訓練データとテストデータに分けるなどの作業を行う

In [28]:
x_train, y_train, x_train_original= [], [], []# x_train_org はログ用
x_test,  y_test,  x_test_original= [], [], []# x_test_org はログ用
n_eachclass_train = 800
n_eachclass_test = 100
n_eachclass = n_eachclass_train + n_eachclass_test
prob = {"cute":[1,0,0],"cool":[0,1,0],"passion":[0,0,1]}
for type_temp in types:
    data_temp = data[data.type == type_temp]#type_tempに一致するタイプのデータをすべて取り出す
    if (data_temp.shape[0] < n_eachclass):
        print(u'データ数が足りません: タイプ' + type_temp + u', ' + str(data_temp.shape[0]))
        sys.exit()
    data_temp = data_temp.loc[random.sample(list(data_temp.index), n_eachclass)]#いま注目しているタイプについてn_eachclass個だけのデータをランダムに取り出す
    original_text = data_temp.iloc[:,max_speech_length+1]#オリジナルのセリフの抽出
    name_text = data_temp.iloc[:,max_speech_length+2]#名前の抽出
    data_temp = data_temp.iloc[:,1:(max_speech_length+1)]#数字としての情報を取り出す
    for i in range(n_eachclass_train):
        x_train.append(data_temp.iloc[i].as_matrix())
        y_train.append(prob[type_temp])
        x_train_original.append([original_text.iloc[i], type_temp,name_text.iloc[i]])
    for i in range(n_eachclass_train, n_eachclass):
        x_test.append(data_temp.iloc[i].as_matrix())
        y_test.append(prob[type_temp])
        x_test_original.append([original_text.iloc[i], type_temp,name_text.iloc[i]])
x_train = numpy.array(x_train)
y_train = numpy.array(y_train)
x_test  = numpy.array(x_test)
y_test  = numpy.array(y_test)

C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


学習の実行、訓練データとテストデータそれぞれに対する精度が出力される

In [29]:
# モデルを訓練します
model.fit(x_train, y_train, nb_epoch=50, batch_size=200)

# 訓練したモデルの性能をテストします
scores = model.evaluate(x_train, y_train, verbose=0)
print("Accuracy (train): %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy (test) : %.2f%%" % (scores[1]*100))

C:\Users\yota-\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/50
2400/2400 [==============================] - 11s 4ms/step - loss: 1.0988 - acc: 0.3392
Epoch 2/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0995 - acc: 0.3346
Epoch 3/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0991 - acc: 0.3329
Epoch 4/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0987 - acc: 0.3325
Epoch 5/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0987 - acc: 0.3446
Epoch 6/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0965 - acc: 0.3488
Epoch 7/50
2400/2400 [==============================] - 3s 1ms/step - loss: 1.0487 - acc: 0.4796
Epoch 8/50
2400/2400 [==============================] - 3s 1ms/step - loss: 0.9707 - acc: 0.5333
Epoch 9/50
2400/2400 [==============================] - 3s 1ms/step - loss: 0.9177 - acc: 0.5642
Epoch 10/50
2400/2400 [==============================] - 3s 1ms/step - loss: 0.8476 - acc: 0.6063
Epoch 11/50
2400/2400 [=====

In [30]:
prob = pandas.DataFrame(model.predict(x_test), columns = types)
for i in range(len(prob)):
    print(prob.iloc[i],x_test_original[i])

cute       0.997134
cool       0.001526
passion    0.001339
Name: 0, dtype: float32 ['馬車に揺られて、のんびりとテーマパーク内を散策も楽しいですわね○○さん。でもこうしていると、まるで本当に西部劇の世界に迷い込んでしまったみたいでドキドキしますわ…うふふ♪', 'cute', '［フリルドウェスタン］相原雪乃']
cute       0.006107
cool       0.000122
passion    0.993772
Name: 1, dtype: float32 ['『はぁぁぁッ、秘剣ファイヤーサークル・クランチーッ！』 空に金環食が輝く時、勇者ノリコの力は完全復活だよっ。聖なる窯火を剣に変えて、悪いオークたちをやっつけちゃうから、覚悟してね！ てやーっ！！', 'cute', '［目覚めし勇者］椎名法子+']
cute       0.997205
cool       0.001444
passion    0.001350
Name: 2, dtype: float32 ['どうかなプロデューサーさん！あずきにドキッとした？ふっふっふ、これなら作戦は成功かなっ。さぁ、あずきたちで温泉街を盛り上げる、名付けて湯けむりアイドル大作戦！作戦開始だよっ！', 'cute', '［湯けむり大作戦］桃井あずき+']
cute       0.509075
cool       0.486080
passion    0.004845
Name: 3, dtype: float32 ['今年のボクは、才能だけじゃなく運気にもあふれているんです！巫女さんとして、ファンのみなさんにも幸せを分けてあげますからね！ご利益があるのは間違いなしです！ ボクの運ですから！！', 'cute', '［自称・幸運］輿水幸子+']
cute       0.997099
cool       0.001598
passion    0.001303
Name: 4, dtype: float32 ['みんなのトークが膨らむたびに、お笑いの花もきれいに咲くっ！ それはまるで、このバルーンフラワーみたいにっ！ ナイストークのみんなに一輪ずつプレゼントしてあげるね。遠慮なんてしちゃDA・ME・DA・

Name: 80, dtype: float32 ['ええと…ですから、クラシックはロックではなくて…ええ、はい。クラシックロックと呼ばれるジャンルはあります…でもクラシック音楽は別で…あの、聞いてくださいませ。…○○様、困りものですわ', 'cute', '［ガールズロッカー］涼宮星花']
cute       0.034416
cool       0.964658
passion    0.000926
Name: 81, dtype: float32 ['LIVEに来てくれたファンのみなさんに…幸せな気持ちを、心に持って帰ってもらいたいです…。こんな私でも、誰かを幸せにできたらうれしいな…。じゃあ…歌います。せーのっ、いぇいっ', 'cute', '［クローバーエンジェル］緒方智絵里+']
cute       0.997215
cool       0.001433
passion    0.001352
Name: 82, dtype: float32 ['はじめまして、○○プロデューサーさん！ 島村卯月、17歳です。私、精一杯頑張りますから、一緒に夢叶えましょうね♪よろしくお願いしますっ！', 'cute', '島村卯月']
cute       0.004170
cool       0.000108
passion    0.995723
Name: 83, dtype: float32 ['やっぱファンのみんなには面白いって思ってもらいたい！だからアタシはこれからもオモシロおかしく、楽しくアイドルするよ！さぁ、○○ちゃんも瑛梨華のエンターテイメントを楽しんで！せーの、イェイ☆', 'cute', '［ラブリー☆バラドル］赤西瑛梨華+']
cute       0.997149
cool       0.001525
passion    0.001326
Name: 84, dtype: float32 ['わぁ、流れ星！ ○○さん、見ました？ 流れ星ですよぉ。天体観測していて見られるなんてラッキーですね。私、もっとキレイになれるようにってお願いしました。うふふ、叶うといいですよねぇ', 'cute', '［ロマンチックナイト］井村雪菜']
cute       0.970733
cool       0.024879
passion    0.

Name: 204, dtype: float32 ['わ～っ♪こんな可愛い衣装、ありがとうプロデューサー！！ 私、何だかやれそうな気がしてきたよー！ えへへ、私、これまで以上にいーっぱい頑張るから、これからもよろしくね！', 'passion', '本田未央+']
cute       0.997184
cool       0.001474
passion    0.001342
Name: 205, dtype: float32 ['この衣装、白くてふわふわで、何だか気分までふわふわ～ってなりますね。えへへっ！自分のCDが出せて、こんな可愛い衣装まで着られて…私、幸せすぎてふわふわ～って…えっ、もう出番！？', 'passion', '［CDデビュー］十時愛梨+']
cute       0.001592
cool       0.998188
passion    0.000221
Name: 206, dtype: float32 ['見よッ！この華麗なるモップさばきッ！プールサイドに、冷たく舞い散る水しぶき、みんなの火照った体をクールダウンッ！マリンボーダーの水兵さんが、アナタの心をガシガシ磨いてピッカピカにするからね', 'passion', '［シューティングチアー］愛野渚+']
cute       0.997167
cool       0.001509
passion    0.001324
Name: 207, dtype: float32 ['お疲れ様です！わたしはお休みだったのですが○○プロデューサーさんに渡したいモノがあったので来ちゃいました。今日はバレンタイン！わたしのチョコで元気になってもらえれば嬉しいです！', 'passion', '［バレンタイン］ルーキートレーナー']
cute       0.003504
cool       0.000106
passion    0.996390
Name: 208, dtype: float32 ['ドイツもコイツも！バレンタイン！コクハクレンアイ至上主義！ボッチたちが！静かに暮らす権利は！ないのか！フ…フヒ…フハハハ！なら…この私がバレンタインデーを真っ黒に染め上げてやる！…っていうの。どう？フヒヒ', 'passion', '［ブラックバレンタイン］星輝子+']
cute    

In [33]:
miss = {}
miss["serif"] = []
miss["type_pred"] = []
miss["type_true"] = []
miss["name"] = []
prob_original = {"cute":np.array([1,0,0]),"cool":np.array([0,1,0]),"passion":np.array([0,0,1])}
def index_to_type(x,types):
    for i in range(len(x)):
        if x[i] == 1:
            return types[i]

for i in range(len(prob)):
    if prob.iloc[i].idxmax(axis = 1) != index_to_type(y_test[i],types):
        miss["serif"].append(x_test_original[i][0])
        miss["type_pred"].append(prob.iloc[i].idxmax(axis = 1))
        miss["type_true"].append(index_to_type(y_test[i],types))
        miss["name"].append(x_test_original[i][2])
        print("prediction:"+prob.iloc[i].idxmax(axis = 1)+" true:"+index_to_type(y_test[i],types))
        print("name:"+x_test_original[i][2]+" serif:"+x_test_original[i][0])

pred:passion true:cute
name:［目覚めし勇者］椎名法子+ serif:『はぁぁぁッ、秘剣ファイヤーサークル・クランチーッ！』 空に金環食が輝く時、勇者ノリコの力は完全復活だよっ。聖なる窯火を剣に変えて、悪いオークたちをやっつけちゃうから、覚悟してね！ てやーっ！！
pred:cool true:cute
name:［クッキングチャレンジ］西園寺琴歌 serif:お料理番組…チャンスをいただけるなら出てみたいですわね。普段お料理はしませんけれど、これまで見て、食べてきた名料理の数々が助けてくれる気がします。見よう見まねで挑戦ですわ！
pred:passion true:cute
name:［おひろめポーズ］赤西瑛梨華+ serif:バッキューーーン☆おおおっ、これ！これだよ、○○ちゃん！NEWバッキューンポーズ、KA・N・SE・I☆ピシッと決まってて、アイドルっぽくて、文句なし夫くん！あちこちのステージやスタジオで決めちゃおー！
pred:cool true:cute
name:［アダルトスタイル］柳清良 serif:この服は白衣…ではありませんね。好きなんです、ベージュ系のワンピースやカーディガン。たとえ勤務中でなくても、なるべく清潔感のある服装を心がけたいですよね。看護師…ううん、アイドルとして
pred:passion true:cute
name:［大傾奇娘］丹羽仁美+ serif:見よーっ、この日のためにあつらえたアイドル装束を！ 降り注ぐ歌の雨をはじき、踊るライバルを振り払って、トップアイドルの本陣に迫っちゃうよ☆ 三国一の粋なアイドルは、この仁美なりー♪
pred:passion true:cute
name:［デビリッシュゴシック］間中美里 serif:素敵な衣装ですねぇ、これ～♪それにぃ、なんか刺激的かもぉ！これ着たら性格まで攻撃的になっちゃかもぉ…○○プロデューサー、チクチク責められちゃうのとかどうですかぁ？
pred:passion true:cute
name:［CDデビュー］宮本フレデリカ+ serif:やっほー、みんなのお待ちかね、フレちゃんだよー。あっ、オンスケだっけ？ じゃあ待ってないかな。でも待っててくれた方がアタシはうれしいなー♪じゃ、待たせすぎないうちにLIVEスタート♪
pred:cool true:c

pred:cute true:cool
name:［魅惑のバニー］水木聖來+ serif:ピンクでファンシーな可愛さと、ちょっと大胆に肌を見せるオトナの色気、またアタシの新しい魅力を見せられるみたい♪○○さんが引き出してくれた魅力だよね！ファンのみんなに伝えたいな！
pred:cute true:cool
name:［ステップバイステップ］白坂小梅 serif:はぁ……はぁ……。○○さん、ちゃんと走れました…。え、えへへ…ち、ちょっと…頑張りすぎちゃった…かも…。で、でも…タイムは、あんまり…気にしちゃだめ…だよ…えへへ…
pred:passion true:cool
name:［イタリアンスタイル］ヘレン+ serif:イタリアの最高級ワインと今の私…どちらの方が上かしら？ 味わいの深さで劣る気はまったくしないわ。さて、この国の人たちに真の最高峰を教えてあげるとしましょう、◯◯
pred:cute true:cool
name:［新春コレクション］成宮由愛+ serif:私が…こんなに大きなLIVEで歌えるなんて…すごく緊張するけど…○○さんがいてくれれば…きっと大丈夫です…。……あ、あの…本番まで…手、握ってもらっても…いいですか…？
pred:cute true:cool
name:［グリッターステージ］二宮飛鳥+ serif:フッ…踊らされてもいいさ。プロデューサー、キミも月夜に踊りたくなる気分の日くらいあるだろう？輝くステージに立ち尽くすマリオネットじゃ一瞬を楽しめないからね。さぁ、ファンがボクたちを待っているよ
pred:cute true:passion
name:［おちゃめなサイキッカー］堀裕子 serif:きょ、今日はプロデューサーにサイキックトレーニングをお見せしよう…！ こうやって念を送ると、一瞬で知恵の輪がですね…。ムーン、ムムムーン、おかしいですね、ちょっと調子が…。これ…ぐぬぬ…固い…
pred:cute true:passion
name:［ハッピー☆クリスマス］城ヶ崎莉嘉 serif:実はずーっと前からママのお手伝いして、お小遣い貯めてたんだっ！○○くんと来るの、すーっごく楽しみだったんだよ！ てコトで、このチケットはアタシからのクリスマスプレゼントだよーっ☆
pred:cute true:passion
name:本田未央+ ser

In [34]:
def type_to_index(type_temp):
    if type_temp == "cute":
        return 0
    elif type_temp == "cool":
        return 1
    elif type_temp == "passion":
        return 2

correlation = np.zeros([3,3])

for i in range(len(prob)):
    correlation[type_to_index(index_to_type(y_test[i],types))][type_to_index(prob.iloc[i].idxmax(axis = 1))] += 1
for i in range(len(correlation)):
    correlation[i] = correlation[i]/sum(correlation[i])
print(correlation)

[[0.68 0.13 0.19]
 [0.32 0.64 0.04]
 [0.46 0.05 0.49]]
